## 3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [6]:
from pymongo import MongoClient
import pandas as pd
import json

In [7]:
client = MongoClient('localhost', 27017)
db = client['hh']
hh_vac = db.vacancies

In [14]:
def add_to_db_only_new_items(vacancy :dict):
    # Подготавливаем даные
    find_condition = {'Ссылка': vacancy['Ссылка']}  # Условие для поиска при обновлении данных
    elements_for_update = vacancy  # Оставшиеся элементы для обновления
    elements_for_update.pop('Ссылка')  # Оставшиеся элементы для обновления
    elements_for_update = {'$set': elements_for_update}
    # Кладем в метод
    result_update = hh_vac.update_one(find_condition, elements_for_update)
    result_matched = result_update.matched_count
    if result_matched == 0:  # Если не было обновлено ни одного элемента
        hh_vac.insert_one(vacancy)
        return 1
    return 0

In [27]:
with open('result.json') as file:
    vacancies = json.loads(file.read())
df = pd.DataFrame(vacancies).fillna('')
# Преобразуем в нужный вид
vacancies = []
for ind in df.index:
    vacancies.append(df.loc[ind].to_dict())

**Переменная "vacancies" - должен быть списом словарей вакансий**

In [28]:
%%time
cnt = 0  # Счетчик 
for vac in vacancies:
    cnt += add_to_db_only_new_items(vacancy=vac)
print(f'Добавлено {cnt} новых элементов')

Добавлено 0 новых элементов
Wall time: 8.83 s
